In [ ]:
# TODO: Use RNN to process each protein and produce a hidden state, maybe use an embedding layer
# normalize for sequence length somehow
# Maybe use LSTM or Transformer instead

In [ ]:
# TODO: Output layer should produce potential 3mers
# linear layer from hidden vector to output classes
# use cross entropy loss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install torchsummaryX -q
!pip install torchsummaryx==1.1.0 -q
!pip install torchmetrics -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 10.3 MB/s eta 0:00:00


In [ ]:
import torch
from torchsummaryX import summary
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import numpy as np
import gc
import os
from tqdm import tqdm
from torchmetrics.classification import MultilabelF1Score, MultilabelAccuracy, MultilabelSpecificity
from sklearn.metrics import precision_score, recall_score

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
!wget -O '/content/train_data.tar.gz' 'https://chemdata.nist.gov/download/peptide_library/libraries/human/ION_TRAP/2014_05_29/2014_05_29_human_consensus_final_true_lib.tar.gz'
!wget -O '/content/valid_data.tar.gz' 'https://chemdata.nist.gov/download/peptide_library/libraries/mouse/ION_TRAP/2013_05_20/2013_05_20_mouse_consensus_final_true_lib.tar.gz'
# !wget -O '/content/valid_data.tar.gz' 'https://chemdata.nist.gov/download/peptide_library/libraries/proteins/ion_trap/bsa/strict/2011_04_01/2011_04_01_bsa_consensus_final_true_lib.tar.gz'

!tar -xvzf '/content/train_data.tar.gz'
!tar -xvzf '/content/valid_data.tar.gz'

--2024-05-03 13:02:46--  https://chemdata.nist.gov/download/peptide_library/libraries/human/ION_TRAP/2014_05_29/2014_05_29_human_consensus_final_true_lib.tar.gz
Resolving chemdata.nist.gov (chemdata.nist.gov)... 129.6.13.19, 2610:20:6005:13::19
Connecting to chemdata.nist.gov (chemdata.nist.gov)|129.6.13.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 573909561 (547M) [application/x-gzip]
Saving to: ‘/content/train_data.tar.gz’

/content/train_data 100%[===================>] 547.32M  12.8MB/s    in 77s     

2024-05-03 13:04:04 (7.09 MB/s) - ‘/content/train_data.tar.gz’ saved [573909561/573909561]

--2024-05-03 13:04:04--  https://chemdata.nist.gov/download/peptide_library/libraries/mouse/ION_TRAP/2013_05_20/2013_05_20_mouse_consensus_final_true_lib.tar.gz
Resolving chemdata.nist.gov (chemdata.nist.gov)... 129.6.13.19, 2610:20:6005:13::19
Connecting to chemdata.nist.gov (chemdata.nist.gov)|129.6.13.19|:443... connected.
HTTP request sent, awaiting response.

In [ ]:
AAs = ['A','R','N','D','C','E','Q','G','H','I','L','K','M','F','P','S','T','W','Y','V']
THREEMERS = []
for i in range(len(AAs)):
  for j in range(len(AAs)):
    for k in range(len(AAs)):
      THREEMERS.append(AAs[i] + AAs[j] + AAs[k])
THREEMER_MAP = dict()
for i in range(len(THREEMERS)):
  THREEMER_MAP[THREEMERS[i]] = i

In [ ]:
# TODO: Dataloader
# Process input files into lists of (mass, intensity) pairs for each protein
# Convert protein sequence into binary vector of 3-mers
class MSDataset(torch.utils.data.Dataset):
  def __init__(self, file, map=THREEMER_MAP):
    with open(file, 'r') as f:
      specs = f.read().strip().split('\n\n')
      self.spectra = []
      self.length = len(specs)
      output_size = len(THREEMER_MAP)
      self.labels = np.zeros((self.length, output_size))
      k = 0

      for spec in specs:
        lines = spec.strip().split('\n')
        seq = lines[0].split('Name: ')[1].split('/')[0]

        threemers = set()
        for i in range(len(seq) - 2):
          threemers.update([seq[i:i+3]])
        for t in threemers:
          self.labels[k, THREEMER_MAP[t]]= 1

        i = 1
        while 'Num peaks: ' not in lines[i]:
          i += 1
        spec_len = int(lines[i].split('Num peaks: ')[1])
        spectrum = np.zeros((spec_len, 2))
        i += 1
        for j in range(i, i + spec_len):
          nums = lines[j].split('\t')
          spectrum[j-i,0] = float(nums[0])
          spectrum[j-i,1] = float(nums[1])
        self.spectra.append(spectrum)
        k += 1
      self.abundance = np.zeros(output_size)
      for i in range(output_size):
        self.abundance[i] = self.length / (np.sum(self.labels[:,i]) + 1e-4)


  def __len__(self):
    return self.length

  def __getitem__(self, ind):
    spec = torch.FloatTensor(self.spectra[ind])
    lbl = torch.tensor(self.labels[ind,:])
    return spec, lbl

  def collate_fn(self, batch):
    batch_spec, batch_lbl = zip(*batch)

    batch_spec_pad = pad_sequence(batch_spec, batch_first=True)
    lengths_spec = [len(seq) for seq in batch_spec]


    return batch_spec_pad, torch.tensor(lengths_spec), torch.stack(batch_lbl, dim=0)

In [ ]:
# RESNET
# Process input files into lists of (mass, intensity) pairs for each protein
# Convert protein sequence into binary vector of 3-mers
class MSDataset(torch.utils.data.Dataset):
  def __init__(self, file, map=THREEMER_MAP, bin_size=0.25, max_m_z = 1000):
    with open(file, 'r') as f:
      specs = f.read().strip().split('\n\n')
      self.spectra = []
      self.length = len(specs)
      output_size = len(THREEMER_MAP)
      self.labels = np.zeros((self.length, output_size))
      k = 0

      for spec in specs:
        lines = spec.strip().split('\n')
        seq = lines[0].split('Name: ')[1].split('/')[0]

        threemers = set()
        for i in range(len(seq) - 2):
          threemers.update([seq[i:i+3]])
        for t in threemers:
          self.labels[k, THREEMER_MAP[t]]= 1

        i = 1
        while 'Num peaks: ' not in lines[i]:
          i += 1
        spec_len = int(lines[i].split('Num peaks: ')[1])

        spectrum = np.zeros((int(max_m_z/bin_size),1))
        i += 1
        max_peak = 0
        min_peak = 100000
        for j in range(i, i+spec_len):
          nums = lines[j].split('\t')
          peak = float(nums[0])
          intensity = float(nums[1])
          index = int(peak/bin_size)
          if intensity > max_peak:
            max_peak = intensity
          if intensity < min_peak:
            min_peak = intensity
          if index < int(max_m_z/bin_size):
            spectrum[index][0] = intensity
        spectrum = (spectrum - min_peak)/(max_peak - min_peak)
        self.spectra.append(spectrum)
        k += 1

      self.abundance = np.zeros(output_size)
      for i in range(output_size):
        self.abundance[i] = self.length / (np.sum(self.labels[:,i]) + 1e-4)


  def __len__(self):
    return self.length

  def __getitem__(self, ind):
    spec = torch.FloatTensor(self.spectra[ind])
    lbl = torch.tensor(self.labels[ind,:])
    return spec, lbl

  def collate_fn(self, batch):
    batch_spec, batch_lbl = zip(*batch)

    batch_spec_pad = pad_sequence(batch_spec, batch_first=True)
    lengths_spec = [len(seq) for seq in batch_spec]


    return batch_spec_pad, torch.tensor(lengths_spec), torch.stack(batch_lbl, dim=0)

In [ ]:
class MSDatasetTest(torch.utils.data.Dataset):
  def __init__(self, file, map=THREEMER_MAP):
    with open(file, 'r') as f:
      specs = f.read().strip().split('\n\n')
      self.spectra = []
      self.length = len(specs)

      for spec in specs:
        lines = spec.strip().split('\n')
        i = 1
        while 'Num peaks: ' not in lines[i]:
          i += 1
        spec_len = int(lines[i].split('Num peaks: ')[1])
        spectrum = np.zeros((spec_len, 2))
        i += 1
        for j in range(i, i + spec_len):
          nums = lines[j].split('\t')
          spectrum[j-i,0] = float(nums[0])
          spectrum[j-i,1] = float(nums[1])
        self.spectra.append(spectrum)


  def __len__(self):
    return self.length

  def __getitem__(self, ind):
    spec = torch.FloatTensor(self.spectra[ind])
    return spec

  def collate_fn(self, batch):
    batch_spec = batch

    batch_spec_pad = pad_sequence(batch_spec, batch_first=True)
    lengths_spec = [len(seq) for seq in batch_spec]


    return batch_spec_pad, torch.tensor(lengths_spec)

In [ ]:
train_file = '/content/human_consensus_final_true_lib.msp'
valid_file = '/content/mouse_consensus_final_true_lib.msp'

# Feel free to add more items here
config = {
    "lr"         : 1e-3,
    "epochs"     : 30,
    "batch_size" : 128  # Increase if your device can handle it
}

dropout = 0.3

In [ ]:
gc.collect()

30

In [ ]:
abundance_train = np.zeros(8000)
train_data = MSDataset(train_file, abundance_train)
print(np.sum(abundance_train == np.zeros(8000)))
print(np.average(abundance_train))

8000
0.0


In [ ]:
# Create objects for the dataset class
train_data = MSDataset(train_file)
abundance_train = train_data.abundance
val_data = MSDataset(valid_file)

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    collate_fn = train_data.collate_fn,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    collate_fn = val_data.collate_fn,
    shuffle     = False
)

# test_loader = torch.utils.data.DataLoader(
#     dataset     = test_data,
#     num_workers = 2,
#     batch_size  = config['batch_size'],
#     pin_memory  = True,
#     collate_fn = test_data.collate_fn,
#     shuffle     = False
# )

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
# print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  128
Train dataset samples = 340356, batches = 2660
Val dataset samples = 149458, batches = 1168


In [ ]:
abundance_train = train_data.abundance
print(np.average(1/abundance_train))

0.072714028125


In [ ]:
# sanity check
for data in train_loader:
    x, lx, y = data
    print(x.shape, lx.shape, y.shape)
    break

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


torch.Size([128, 4000, 1]) torch.Size([128]) torch.Size([128, 8000])


In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
class ResBlock(torch.nn.Module):
    def __init__(self, in_chan, out_chan):
        super().__init__()

        stride = 1
        self.convert = (in_chan != out_chan)
        if self.convert:
          stride = 2
          self.increase = torch.nn.Conv1d(in_chan, out_chan, 1, stride = stride)

        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv1d(in_chan, out_chan, 3, padding = 1, stride = stride),
            torch.nn.Dropout(p=dropout, inplace=True),
            torch.nn.BatchNorm1d(out_chan),
            torch.nn.ReLU(inplace=True)
            )
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv1d(out_chan, out_chan, 3, padding = 1),
            # torch.nn.Dropout(p=dropout, inplace=True),
            torch.nn.BatchNorm1d(out_chan)
            )
        self.relu = torch.nn.ReLU(inplace=True)

    def forward(self, x):
      resid = x
      if self.convert:
        resid = self.increase.forward(x)
      out = self.conv1.forward(x)
      out = self.conv2.forward(out)
      out += resid
      return self.relu.forward(out)

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size, cnn_size):
        super(Encoder, self).__init__()

        self.embedding = torch.nn.Sequential(
            torch.nn.Conv1d(in_channels=input_size, out_channels=64, kernel_size=7, stride = 3, padding=7//2),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
        )
        self.pool = torch.nn.Sequential(torch.nn.MaxPool1d(kernel_size = 3, stride = 1))
        self.backbone = torch.nn.Sequential(
          ResBlock(64, 64),
          ResBlock(64, 64),
          # ResBlock(64, cnn_size)
          ResBlock(64, 128),
          ResBlock(128, 128),
          ResBlock(128, 256),
          # ResBlock(256, cnn_size)
          ResBlock(256, 256),
          ResBlock(256, 512),
          ResBlock(512, cnn_size)
        )

    def forward(self, x, x_lens):
        x = x.transpose(1,2)
        out = self.embedding(x)
        out = self.pool(out)
        out = self.backbone(out)
        x = x.transpose(1,2)
        out = torch.reshape(out, (out.shape[0], -1))

        return out

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()

        cnn_size = encoder_hidden_size

        self.embedding = torch.nn.Sequential(
            PermuteBlock(), torch.nn.Conv1d(in_channels=input_size, out_channels=cnn_size//2, kernel_size=3, padding=1), PermuteBlock(),
            torch.nn.ReLU(),
            PermuteBlock(), torch.nn.Conv1d(in_channels=cnn_size//2, out_channels=cnn_size, kernel_size=3, padding=1), PermuteBlock(),
            torch.nn.ReLU(),
            # PermuteBlock(), torch.nn.Conv1d(in_channels=cnn_size//4, out_channels=cnn_size//2, kernel_size=3, padding=1), PermuteBlock(),
            # torch.nn.ReLU(),
            # PermuteBlock(), torch.nn.Conv1d(in_channels=cnn_size//2, out_channels=cnn_size, kernel_size=3, padding=1), PermuteBlock(),
            # torch.nn.ReLU(),
            # PermuteBlock(), torch.nn.Conv1d(in_channels=cnn_size//2, out_channels=cnn_size, kernel_size=5, padding=2), PermuteBlock()
            # torch.nn.ReLU(),
            # PermuteBlock(), torch.nn.AvgPool1d(kernel_size=3, padding = 1), PermuteBlock()
            # PermuteBlock(), torch.nn.Conv1d(in_channels=cnn_size//2, out_channels=cnn_size, kernel_size=5, padding=2), PermuteBlock(),
            # torch.nn.ReLU(),
            # PermuteBlock(), torch.nn.MaxPool1d(kernel_size=5, padding = 2), PermuteBlock()
        )
        # self.rnn1 = torch.nn.RNN(input_size = cnn_size, num_layers=4, hidden_size = encoder_hidden_size//2, bidirectional=True, batch_first=True, dropout=0.3)
        # self.rnn2 = torch.nn.RNN(input_size = encoder_hidden_size, hidden_size = encoder_hidden_size//2, bidirectional=True, batch_first=True)
        self.rnn1 = torch.nn.LSTM(input_size = cnn_size, num_layers=4, hidden_size = encoder_hidden_size//2, bidirectional=True, batch_first=True, dropout=0.3)
        self.rnn2 = torch.nn.LSTM(input_size = encoder_hidden_size, hidden_size = encoder_hidden_size//2, bidirectional=True, batch_first=True)

    def forward(self, x, x_lens):
        x = self.embedding(x)
        # TODO: Pack Padded Sequence
        x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        x_packed, _ = self.rnn1(x_packed)
        _, out = self.rnn2(x_packed)
        out, _ = out
        out = torch.reshape(out, (out.shape[0]//2, out.shape[1], out.shape[2]*2))
        out = torch.squeeze(out)

        return out

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 8000):
        super().__init__()

        factor = 84 # for 256

        self.mlp = torch.nn.Sequential(
            torch.nn.BatchNorm1d(embed_size*factor),
            torch.nn.Linear(embed_size*factor, 1024),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.3, inplace=True),
            torch.nn.BatchNorm1d(1024),
            torch.nn.Linear(1024, 2048),
            torch.nn.GELU(),
            torch.nn.Dropout(p=0.3, inplace=True),
            torch.nn.BatchNorm1d(2048),
            torch.nn.Linear(2048, output_size)
        )

        # self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, encoder_out):
        out = self.mlp(encoder_out)
        # out = self.softmax(out)
        return out

In [ ]:
class MSModel(torch.nn.Module):

    def __init__(self, input_size, cnn_size = 256, embed_size= 1024, output_size= 8000):
        super().__init__()

        self.encoder        = Encoder(input_size, embed_size, cnn_size)
        self.decoder        = Decoder(embed_size, output_size)



    def forward(self, x, lengths_x):

        encoder_out  = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out

In [ ]:
model = MSModel(
    input_size  = 1,
    cnn_size = 256,
    embed_size  = 256
).to(device)

# model = MSModel(
#     input_size  = 2,
#     embed_size  = 128
# ).to(device)
# print(model)
# summary(model, x.to(device), lx)

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss(pos_weight = torch.tensor(abundance_train).to(device))

optimizer =  torch.optim.Adam(model.parameters(), config['lr'], weight_decay=0.1)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 3, factor=0.7, threshold=0.1)

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

In [ ]:
def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')
    metric = MultilabelF1Score(num_labels=8000).to(device)
    metric2 = MultilabelAccuracy(num_labels=8000).to(device)
    metric3 = MultilabelSpecificity(num_labels=8000).to(device)

    total_loss = 0
    tf = 0
    # tacc = 0
    taccs = 0
    prec = 0
    rec = 0
    spec = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, lx, y = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h = model(x, lx)
            loss = criterion(h, y)

        total_loss += loss.item()
        tf += metric(h, y).item()
        taccs += metric2(h, y).item()
        spec += metric3(h, y).item()
        h = torch.sigmoid(h)
        predictions = torch.round(h)

        y = y.detach().cpu().numpy()
        predictions = predictions.detach().cpu().numpy()

        prec += precision_score(y, predictions, average='macro', zero_division=0)
        rec += recall_score(y, predictions, average='macro', zero_division=0)
        # tacc    += torch.sum(predictions == y).item()/(h.shape[0]*8000)

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            acc_s="{:.04f}%".format(float(taccs*100 / (i + 1))),
            f="{:.04f}%".format(float(tf / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, h, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader), tf / len(train_loader), taccs / len(train_loader), prec / len(train_loader), rec  / len(train_loader), spec  / len(train_loader)


def validate_model(model, val_loader):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
    metric = MultilabelF1Score(num_labels=8000).to(device)
    metric2 = MultilabelAccuracy(num_labels=8000).to(device)
    metric3 = MultilabelSpecificity(num_labels=8000).to(device)

    total_loss = 0
    vf = 0
    # vacc = 0
    vaccs = 0
    prec = 0
    rec = 0
    spec = 0

    for i, data in enumerate(val_loader):

        x, lx, y = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h = model(x, lx)
            loss = criterion(h, y)

        total_loss += loss.item()
        vf += metric(h, y).item()
        vaccs += metric2(h, y).item()
        spec += metric3(h, y).item()
        h = torch.sigmoid(h)
        predictions = torch.round(h)

        y = y.detach().cpu().numpy()
        predictions = predictions.detach().cpu().numpy()

        prec += precision_score(y, predictions, average='macro', zero_division=0)
        rec += recall_score(y, predictions, average='macro', zero_division=0)
        # vf += metric(predictions, y).item()
        # vacc    += torch.sum(predictions == y).item()/(h.shape[0]*8000)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))),
                              acc="{:.04f}%".format(float(vaccs*100 / (i + 1))),
                              f="{:.04f}%".format(float(vf / (i + 1))))

        batch_bar.update()

        del x, y, lx, h, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    vf /= len(val_loader)
    vaccs /= len(val_loader)
    prec /= len(val_loader)
    rec /= len(val_loader)
    spec /= len(val_loader)
    return total_loss, vf, vaccs, prec, rec, spec

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_f', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
best_model_path = '/content/drive/MyDrive/best_MS_RNN_model.pth'
best_f = 0

In [ ]:
torch.cuda.empty_cache()
gc.collect()

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_loss, train_f, train_acc, precision_t, recall_t, spec_t   = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_f, valid_acc, precision_v, recall_v, spec_v  = validate_model(model, val_loader)
    scheduler.step(valid_loss)

    print("\tTrain F1 {:.04f}\tTrain Acc {:.04f}\t Train Loss {:.04f}\t Train Prec {:.04f}\t Train Rec {:.04f}\t Train Spec {:.04f}\t Learning Rate {:.07f}".format(train_f, train_acc, train_loss, precision_t, recall_t, spec_t, curr_lr))
    print("\tVal F1 {:.04f}\tVal Acc {:.04f}\t Val Loss {:.04f}\t Val Prec {:.04f}\t Val Rec {:.04f}\t Val Spec {:.04f}".format(valid_f, valid_acc, valid_loss, precision_v, recall_v, spec_v))



    # if valid_f >= best_f:
    #     best_f = valid_f
    #     save_model(model, optimizer, scheduler, ['valid_f', valid_f], epoch, best_model_path)
    #     print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models


Epoch: 1/30


Train:   0%|          | 0/2660 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Val:   0%|          | 0/1168 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


	Train F1 0.0017	Train Acc 0.7361	 Train Loss 1.3861	 Train Prec 0.0009	 Train Rec 0.0463	 Train Spec 0.7369	 Learning Rate 0.0010000
	Val F1 0.0014	Val Acc 0.5905	 Val Loss 15.2030	 Val Prec 0.0007	 Val Rec 0.0412	 Val Spec 0.5907

Epoch: 2/30


Train:   0%|          | 0/2660 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Val:   0%|          | 0/1168 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


	Train F1 0.0003	Train Acc 0.8707	 Train Loss 1.3836	 Train Prec 0.0001	 Train Rec 0.0149	 Train Spec 0.8721	 Learning Rate 0.0010000
	Val F1 0.0015	Val Acc 0.5932	 Val Loss 15.2030	 Val Prec 0.0008	 Val Rec 0.0428	 Val Spec 0.5934

Epoch: 3/30


...

Train:   0%|          | 0/2660 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Val:   0%|          | 0/1168 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


	Train F1 0.0000	Train Acc 0.9709	 Train Loss 1.3837	 Train Prec 0.0000	 Train Rec 0.0011	 Train Spec 0.9726	 Learning Rate 0.0001681
	Val F1 0.0015	Val Acc 0.5474	 Val Loss 15.2030	 Val Prec 0.0008	 Val Rec 0.0435	 Val Spec 0.5475

Epoch: 25/30


Train:   0%|          | 0/2660 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Val:   0%|          | 0/1168 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


	Train F1 0.0000	Train Acc 0.9709	 Train Loss 1.3837	 Train Prec 0.0000	 Train Rec 0.0010	 Train Spec 0.9725	 Learning Rate 0.0001681
	Val F1 0.0016	Val Acc 0.5505	 Val Loss 15.2030	 Val Prec 0.0008	 Val Rec 0.0446	 Val Spec 0.5506

Epoch: 26/30


Train:   0%|          | 0/2660 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Train:  44%|████▍     | 1174/2660 [10:58<13:46,  1.80it/s, acc_s=98.1340%, f=0.0000%, loss=1.3833, lr=0.000118]